In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define paths
train_data_dir = 'Covid/train'
test_data_dir = 'Covid/test'

# Define image size and batch size
img_width, img_height = 150, 150
batch_size = 32

# Create data generators with data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only rescale for the validation/test set (no data augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of augmented data for training and validation
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='grayscale',  # Set color_mode to 'grayscale'
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='grayscale',  # Set color_mode to 'grayscale'
    class_mode='categorical',
    shuffle=False  # No shuffling for the test set
)

# Build a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),  # Set input_shape to (img_width, img_height, 1)
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # Assuming 3 classes: covid, normal, viral
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)
# Save the model
model.save("covid_classifier_model.h5")

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Accuracy: {test_acc}')



Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.



Epoch 1/10


7/7 [==============================] - 39s 5s/step - loss: 1.2205 - accuracy: 0.4475 - val_loss: 1.0726 - val_accuracy: 0.6406
Epoch 2/10
7/7 [==============================] - 24s 3s/step - loss: 0.9953 - accuracy: 0.5799 - val_loss: 0.9137 - val_accuracy: 0.5469
Epoch 3/10
7/7 [==============================] - 25s 4s/step - loss: 0.7074 - accuracy: 0.6920 - val_loss: 0.5112 - val_accuracy: 0.7188
Epoch 4/10
7/7 [==============================] - 24s 3s/step - loss: 0.4944 - accuracy: 0.7671 - val_loss: 0.5397 - val_accuracy: 0.7031
Epoch 5/10
7/7 [==============================] - 24s 4s/step - loss: 0.4724 - accuracy: 0.7900 - val_loss: 0.2950 - val_accuracy: 0.8906
Epoch 6/10
7/7 [==============================] - 26s 4s/step - loss: 0.5118 - accuracy: 0.7808 - val_loss: 0.5921 - val_accuracy: 0.6250
Epoch 7/10
7/7 [==============================] - 26s 4s/step - loss: 0.3961 - accura

C:\Users\92316\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 3s 2s/step - loss: 0.3073 - accuracy: 0.8438
Test Accuracy: 0.84375


In [5]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('covid_classifier_model.h5')  # Replace 'your_model_path.h5' with the actual path to your saved model

# Load and preprocess the new image
img_path = 'covid_img.jpg'  # Replace 'path_to_your_image.jpg' with the actual path to your image
img = image.load_img(img_path, target_size=(150, 150), color_mode='grayscale')  # Assuming the model was trained on grayscale images
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize pixel values

# Make a prediction
predictions = model.predict(img_array)

# Decode the predictions to get the class label
class_index = np.argmax(predictions)
if class_index == 0:
    predicted_class = 'Covid'
elif class_index == 1:
    predicted_class = 'Normal'
else:
    predicted_class = 'Viral'

print("Predicted Class:", predicted_class)


1/1 [==============================] - 0s 133ms/step
Predicted Class: Normal
